In [40]:
from util.utils import *
import os

In [41]:
## load datasets
def loadDataframes():
    cat62 = None
    #cat62 = openDataframe("cat62_bronze.csv")
    espera = openDataframe("espera_bronze.csv")
    bimtra = openDataframe("bimtra_bronze.csv")
    metaf = openDataframe("metaf_bronze.csv")
    metar = openDataframe("metar_bronze.csv")
    tc_prev = openDataframe("tc-prev_bronze.csv")
    tc_real = openDataframe("tc-real_bronze.csv")
    return cat62, espera, bimtra, metaf, metar, tc_prev, tc_real

cat62, espera, bimtra, metaf, metar, tc_prev, tc_real = loadDataframes()

In [42]:
tc_real['aero'] = tc_real['aero'].apply(lambda x: 'SB' + x)

In [43]:
# MERGE BIMTRA - TC_REA

resultado = bimtra.merge(tc_real, left_on=['origem'], right_on=['aero'], how='inner')

# Use uma função lambda para calcular a diferença entre dt_dep e hora
resultado = resultado[resultado.apply(lambda row: abs(row['dt_dep'] - row['hora']) <= 1800000, axis=1)]

resultado = resultado.rename(columns={'hora': 'hora_real'})
resultado = resultado.rename(columns={'aero': 'aero_real'})
resultado = resultado.rename(columns={'hora_formatted': 'hora_formatted_real'})



In [44]:
# MERGE COM RESULTADO E TC PREAV AGORA
tc_prev['aero'] = tc_prev['aero'].apply(lambda x: 'SB' + x)

In [45]:
resultado2 = resultado.merge(tc_prev, left_on=['destino'], right_on=['aero'], how='inner')

# Use uma função lambda para calcular a diferença entre dt_dep e hora
resultado2 = resultado2[resultado2.apply(lambda row: abs(row['hora'] - row['dt_arr']) <= 1800000, axis=1)]
resultado2 = resultado2.rename(columns={'hora': 'hora_prev'})
resultado2 = resultado2.rename(columns={'hora_formatted': 'hora_formatted_prev'})
resultado2 = resultado2.rename(columns={'aero': 'aero_prev'})

In [46]:
## RESULTADO COM METAR
resultado3 = resultado2.merge(metar, left_on=['origem'], right_on=['aero'], how='inner')

# Use uma função lambda para calcular a diferença entre dt_dep e hora
resultado3 = resultado3[resultado3.apply(lambda row: abs(row['dt_dep'] - row['hora']) <= 600000, axis=1)]
resultado3 = resultado3.rename(columns={'hora': 'hora_metar'})
resultado3 = resultado3.rename(columns={'hora_formatted': 'hora_formatted_metar'})
resultado3 = resultado3.rename(columns={'aero': 'aero_metar'})


In [47]:
resultado3.shape

(3676, 20)

In [48]:
## RESULTADO COM METAF
resultado4 = resultado3.merge(metaf, left_on=['destino'], right_on=['aero'], how='inner')

# Use uma função lambda para calcular a diferença entre dt_dep e hora
resultado4 = resultado4[resultado4.apply(lambda row: abs(row['hora'] - row['dt_arr']) <= 600000, axis=1)]
resultado4 = resultado4.rename(columns={'hora': 'hora_metaf'})
resultado4 = resultado4.rename(columns={'hora_formatted': 'hora_formatted_metaf'})
resultado4 = resultado4.rename(columns={'aero': 'aero_metaf'})


In [49]:
resultado4.shape

(832, 24)

In [52]:
resultado4.head()

,flightid,origem,destino,dt_dep,dt_arr,dt_dep_formatted,dt_arr_formatted,hora_real,nova_cabeceira,antiga_cabeceira,...,aero_prev,hora_formatted_prev,hora_metar,metar,aero_metar,hora_formatted_metar,hora_metaf,metaf,aero_metaf,hora_formatted_metaf
2214,810f965ce283a33ab6cbb303ccaaaddd,SBGR,SBBR,1680295888000,1680300446000,"Friday, March 31, 2023 17:51:28","Friday, March 31, 2023 19:07:26",1680295644000,10R,30,...,SBBR,"Friday, March 31, 2023 19:00:00",1680296400000,METAR SBGR 312100Z 07007KT 7000 SCT011 21/20 Q...,SBGR,"Friday, March 31, 2023 18:00:00",1680300000000,METAF SBBR 312200Z 15006KT CAVOK 28/10 Q1013=\n,SBBR,"Friday, March 31, 2023 19:00:00"
7927,d1e8b77083f797ad9f69ebf4d2df0e13,SBKP,SBBR,1680126618000,1680130843000,"Wednesday, March 29, 2023 18:50:18","Wednesday, March 29, 2023 20:00:43",1680126051000,15,17,...,SBBR,"Wednesday, March 29, 2023 20:00:00",1680127200000,METAR SBKP 292200Z VRB02KT 8000 -TSRA FEW025 F...,SBKP,"Wednesday, March 29, 2023 19:00:00",1680130800000,METAF SBBR 292300Z 09004KT CAVOK 26/09 Q1014=\n,SBBR,"Wednesday, March 29, 2023 20:00:00"
28667,5b515ea8e42b84f9f3aa57fdd28d6937,SBPA,SBBR,1679907100000,1679914554000,"Monday, March 27, 2023 05:51:40","Monday, March 27, 2023 07:55:54",1679908025000,11,29,...,SBBR,"Monday, March 27, 2023 08:00:00",1679907600000,METAR SBPA 270900Z 00000KT CAVOK 18/16 Q1017=,SBPA,"Monday, March 27, 2023 06:00:00",1679914800000,METAF SBBR 271100Z 12005KT CAVOK 21/12 Q1016=\n,SBBR,"Monday, March 27, 2023 08:00:00"
50851,558f95b81bae1856ba3d589bd0218de5,SBSP,SBBR,1680283516000,1680289041000,"Friday, March 31, 2023 14:25:16","Friday, March 31, 2023 15:57:21",1680283058000,17R,28,...,SBBR,"Friday, March 31, 2023 16:00:00",1680283500000,SPECI SBSP 311725Z VRB04KT 5000 TSRA BR FEW011...,SBSP,"Friday, March 31, 2023 14:25:00",1680289200000,METAF SBBR 311900Z 12006KT CAVOK 29/09 Q1012=\n,SBBR,"Friday, March 31, 2023 16:00:00"
51171,558f95b81bae1856ba3d589bd0218de5,SBSP,SBBR,1680283516000,1680289041000,"Friday, March 31, 2023 14:25:16","Friday, March 31, 2023 15:57:21",1680285028000,17R,12,...,SBBR,"Friday, March 31, 2023 16:00:00",1680283500000,SPECI SBSP 311725Z VRB04KT 5000 TSRA BR FEW011...,SBSP,"Friday, March 31, 2023 14:25:00",1680289200000,METAF SBBR 311900Z 12006KT CAVOK 29/09 Q1012=\n,SBBR,"Friday, March 31, 2023 16:00:00"


In [51]:
resultado4.to_csv("../data/processed_data/merged_bronze.csv")